In [ ]:
import glob
from pathlib import Path
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer
import shutil
import progressbar as pb

dataset_path = Path('/home/szymswiat/datasets/nih_dataset')
data_entry_df = pd.read_csv(Path('../ds/Data_Entry_2017.csv'))
target_dir = Path('/home/szymswiat/datasets/nih_dataset/filtered')

In [ ]:
all_image_paths = glob.glob(f'{dataset_path}/images_*/images/*.png', recursive=True)
all_image_paths.sort()
all_image_paths = {Path(x).name: Path(x).relative_to(dataset_path) for x in all_image_paths}

data_entry_df['Finding Labels'] = data_entry_df['Finding Labels'].map(lambda x: x.replace('|', ','))

classes = np.unique(list(chain(*data_entry_df['Finding Labels'].map(lambda x: x.split(',')).tolist())))
classes = list(sorted([x for x in classes if len(x) > 0]))


encoder = MultiLabelBinarizer(classes=classes)
labels_str = [c.split(',') for c in list(data_entry_df['Finding Labels'])]
labels_all = encoder.fit_transform(labels_str)

all_df = pd.DataFrame()

all_df['Image_Index'] = data_entry_df['Image Index']
all_df['Image_Path'] = all_df['Image_Index'].map(lambda img_idx: all_image_paths.get(img_idx, ''))
# all_df['Label'] = labels_all.tolist()
all_df['Label_Str'] = labels_str
all_df['Patient_ID'] = data_entry_df['Patient ID']

In [ ]:
card_df = all_df[all_df.apply(lambda x: 'Cardiomegaly' in x['Label_Str'], axis=1)]
no_fin_df = all_df[all_df.apply(lambda x: 'No Finding' in x['Label_Str'], axis=1)]

card_df = card_df.sample(2000, random_state=0)
no_fin_df = no_fin_df.sample(2000, random_state=0)

In [ ]:

for path in pb.progressbar(card_df.append(no_fin_df)['Image_Path']):
    if path == '':
        continue
    path = Path(path)
    target = target_dir / path
    target.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(dataset_path / path, target)

card_df.to_csv(target_dir / 'card_df.csv')
no_fin_df.to_csv(target_dir / 'no_fin_df.csv')